#Sistemas de recomendación y Filtrado Colaborativo

Importar las librerías necesarias para realizar el las operaciones de filtrado colaborativo.

In [4]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

!pip install surprise
from surprise import KNNWithMeans, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split

     |████████████████████████████████| 11.8 MB 6.9 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1623221 sha256=3b6272b79e5e52aea426fc2ac1993b67e1d605a6f003e1fc7d7ddab69cd933a8
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


## Método 1

Esta función normalizará la entrada pred_ratings parámetros:
  pred_ratings (calificaciones de predicción)



In [5]:
def normalize(pred_ratings):
  return (pred_ratings - pred_ratings.min()) / (pred_ratings.max() - pred_ratings.min())

Esta función calculará la descomposición de valor único de la matriz de entrada
  dados n_factores. Luego generará y normalizará las predicciones de calificación de los usuarios.

  **Parámetros:**

*   **mat (CSR Matrix)**: matriz scipy csr correspondiente a la tabla dinámica (pt_df) 
*   **pt_df (DataFrame)**: marco de datos de pandas que es una tabla dinámica
*   **n_factors (Integer)** : Número de valores singulares y vectores a calcular.


Debe ser 1<= n_factors < min(mat.shape).

In [6]:
def generate_prediction_df(mat, pt_df, n_factors):

  if not 1 <= n_factors < min(mat.shape):
   raise ValueError("Must be 1 <= n_factors < min(mat.shape)")
  
  # Factorización de matrices
  u, s, v = svds(mat, k = n_factors)
  s = np.diag(s)

  # Calcular las calificaciones de pred
  pred_ratings = np.dot(np.dot(u, s), v)
  pred_ratings = normalize(pred_ratings)

  # convertir a df
  pred_df = pd.DataFrame(pred_ratings,columns = pt_df.columns, index = list(pt_df.index)).transpose()

  return pred_df

Dado un usr_id y pred_df, esta función recomendará elementos al usuario.
     
**Parámetros:**
      
*   **pred_df (DataFrame)**: generado a partir de la función generate_prediction_df
*   **usr_id (Integer)** : El usuario para el que desea obtener recomendaciones de artículos
*   **n_recs (Integer)** : El número de recomendaciones que desea para este usuario



In [7]:
def recommend_items(pred_df, usr_id, n_recs):

  usr_pred = pred_df[usr_id].sort_values(ascending = False).reset_index().rename(columns = {usr_id : 'sim'})
  rec_df = usr_pred.sort_values(by = 'sim', ascending = False).head(n_recs)
  return rec_df

In [8]:
if __name__ == '__main__':
  # constantes
  PATH = 'data.csv'
  #importar datos
  df = pd.read_csv(PATH)
  print(df.shape)
  # generar una tabla dinámica con lectores en el índice y libros en la columna y los valores sean las calificaciones
  pt_df = df.pivot_table(columns = 'book_id',index = 'reader_id', values = 'book_rating').fillna(0)
  # convertir a una matriz csr
  mat = pt_df.values
  mat = csr_matrix(mat)
  pred_df = generate_prediction_df(mat, pt_df, 10)
  # generar recomendaciones
  print(recommend_items(pred_df, 1000, 10))

(100000, 10)
   book_id       sim
0     1341  0.153466
1     1516  0.148940
2     1839  0.147120
3     1108  0.145674
4     1025  0.145550
5     1824  0.145522
6       60  0.144977
7     2881  0.144857
8     1208  0.144673
9     1897  0.144514


## Método 2

In [18]:
pt_df = df.pivot_table(
    columns = 'book_id', 
    index = 'author_id', 
    values = 'book_rating'
).fillna(0)

Esta función encontrará lectores similares al lector_id que se indica como parámetro

In [19]:
def find_similar_readers(pt_df, reader_id, n_recs):
    '''
    This function will find similar readers to the user specified reader_id
    '''
    
    # lector separado de interés y todos los demás lectores
    reader = pt_df[pt_df.index == reader_id]
    other_readers = pt_df[pt_df.index != reader_id]

    # obtener la similitud del lector actual y todos los demás lectores
    sim = cosine_similarity(reader, other_readers)[0].tolist()
    idx = other_readers.index.tolist()

    # crear un diccionario de similitud para este usuario con todos los demás usuarios
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)

    similar_readers = idx_sim[:n_recs]
    readers = [rdr[0] for rdr in similar_readers]

    return readers

In [20]:
find_similar_readers(pt_df = pt_df, reader_id = 226, n_recs = 5)

[319, 191, 145, 162, 212]

## Método 3

In [21]:
rdf = df[['reader_id', 'book_id', 'book_rating']]

In [22]:
#Leer el conjunto de datos
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rdf, reader)

In [23]:
# Dividir el conjunto de datos
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [25]:
# Usar verdadero/falso basado en el usuario para cambiar entre filtrado colaborativo basado en usuario o basado en elementos
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [26]:
# ejecutar el modelo entrenado contra el conjunto de pruebas
test_pred = algo.test(testset)

In [ ]:
# obtener RMSE
accuracy.rmse(test_pred, verbose=True)

In [27]:
algo.predict(uid = 10, iid = 43)

Prediction(uid=10, iid=43, r_ui=None, est=5, details={'actual_k': 0, 'was_impossible': False})